In [52]:
# 2D Lid-driven Cavity
# VECTOR-POTENTIAL VORTICITY FORMULATION
# Written by Mr A. J. Brierley
# Cranfield University, Bedfordshire, UK
# 03/06/2025


In [53]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
plt.rcParams['animation.html'] = 'html5'

In [54]:
nx = 13
ny = 13
nz = 13
lx = 1.0
ly = 1.0
lz = 1.0
dx = lx/(nx-1)
dy = ly/(ny-1)
dz = lz/(nz-1)

In [55]:
Ut = 3.2 # top wall velocity

In [56]:
# Specify initial values for the streamfunction (psi) and vorticity (Omega)
# at t = 0 on the whole domain
# Then specify conditions that will produce values at the boundaries at t = 0
# And be enforced as the solution is marched through time
psix = np.zeros([nx,ny,nz])
psiy = np.zeros([nx,ny,nz])
psiz = np.zeros([nx,ny,nz])

u = np.zeros([nx,ny,nz])
v = np.zeros([nx,ny,nz])
w = np.zeros([nx,ny,nz])

omega = np.zeros([nx,ny,nz])

In [57]:
#------------------------------------------------------
# VECTOR-POTENTIAL BOUNDARY CONDITIONS
#------------------------------------------------------
# Left wall
for j in range(ny):     # Not inclusive
    for k in range(nz):
        psix[0,j,k] = psix[1,j,k]
        psiy[0,j,k] = 0.0
        psiz[0,j,k] = 0.0

# Bottom wall
for i in range(nx):
    for z in range(nz):
        psix[i,0,k] = 0.0
        psiy[i,0,k] = psiy[i,0,k]
        psiz[i,0,k] = 0.0

# Right wall 
for j in range(ny):
    for z in range(nz):
        psix[nx-1,j,k] = psix[nx-2,j,k]  # Here, nx will try to index 13 which is outside the 0 to 13 exclusive range
        psiy[nx-1,j,k] = 0.0
        psiz[nx-1,j,k] = 0.0

# Top wall
for i in range(nx):
    for z in range(nz):
        psix[i,ny-1,k] = Ut
        psiy[i,ny-1,k] = psiy[i,ny-2,k]
        psiz[i,ny-1,k] = 0.0

# Front wall
for i in range(nx):
    for j in range(ny):
        psix[i,j,0] = 0.0
        psiy[i,j,0] = 0.0
        psiz[i,j,0] = psiz[i,j,1]

# Back wall
for i in range(nx):
    for j in range(ny):
        psix[i,j,nz-1] = 0.0
        psiy[i,j,nz-1] = 0.0
        psiz[i,j,nz-1] = psiz[i,j,nz-2]

In [78]:
#------------------------------------------------
# VORTICITY BOUNDARY CONDITIONS 
#------------------------------------------------
# Left wall
for j in range(ny):
    for k in range(nz):
        omega[0,j,k] = (v[1,j,k]-w[1,j,k])/dx
# Right wall
for j in range(ny):
    for k in range(nz):
        omega[nx-1,j,k] = (v[nx-2,j,k] - w[nx-2,j,k])/dx
# Lower wall 
for i in range(nx):  # starting from 0 and exclusive
    for k in range(nz):
        omega[i,0,k] = (u[i,1,k]-w[i,1,k])/dy
# Front wall
for i in range(nx):
    for j in range(ny):
        omega[i,j,0] = (v[i,j,1] + u[i,j,1])/dz
# Back wall 
for i in range(nx):
    for j in range(ny):
        omega[i,j,nz-1] = (v[i,j,nz-2] - u[i,j,nz-2])/dx
# Top wall 
for i in range(nx):
    for k in range(nz-1):  # NOTE: dodgy bit
        omega[i,ny-1,k] = (u[i,ny-2,k] + u[i,ny-1,k])/dy + (u[i,ny-1,k+1] - u[i,ny-1,k-1])/(2*dz) - w[i,ny-1,k]/dy

In [ ]:
# # Solution storage
# psisol = []
# psisol.append(psi0)
# wsol = []
# wsol.append(w0)

In [ ]:
# # simulation parameters
# beta = 1.5
# tol = 1e-3
# maxIt = 30

# t = 0.0  # initial time and time counter
# v = 0.05  # nu
# dt = min(0.25*dx*dx/v, 4*v/Ut/Ut)
# tend = 1000*dt
# print('dt =', dt, 's')
# print('Re =', Ut*lx/v)


In [ ]:
# while t < tend:
    
#     #------------------------------------
#     # STREAMFUNCTION-POISSON EQUATION
#     #------------------------------------
    
#     it = 0
#     err = 1e5
#     wn = wsol[-1]
#     psi = psisol[-1].copy()
#     while err > tol and it < maxIt:
#         psik = np.zeros_like(psi)
#         psik[1:-1, 1:-1] = psi[1:-1, 1:-1]
#         for i in range(1,nx-1):
#             for j in  range(1,ny-1):
#                 rhs = (dx*dy)**2*wn[j,i] + dy**2*(psi[j,i+1]+ psi[j,i-1]) + dx**2*(psi[j+1,i] + psi[j-1,i])
#                 rhs *= beta/2.0/(dx**2 + dy**2)
#                 psi[j,i] = rhs + (1 - beta)*psi[j,i]
#         err = np.linalg.norm(psi.ravel() - psik.ravel())
#         it += 1
#         # print(it)
#     psisol.append(psi)

#     w = np.zeros_like(wn)

#     #-------------------------------------
#     # 2D VORTICITY TRANSPORT EQUATION
#     #-------------------------------------
    
#     Cx = -(psi[2:,1:-1] - psi[:-2,1:-1])/2.0/dy * (wn[1:-1,2:] - wn[1:-1,:-2])/2.0/dx
#     Cy = (psi[1:-1,2:] - psi[1:-1,:-2])/2.0/dx * (wn[2:,1:-1] - wn[:-2,1:-1])/2.0/dy
#     Dx = (wn[1:-1,2:] + wn[1:-1,:-2] - 2.0*wn[1:-1,1:-1])/dx/dx
#     Dy = (wn[2:,1:-1] + wn[:-2,1:-1] - 2.0*wn[1:-1,1:-1])/dy/dy

#     rhs = Cx + Cy + v*(Dx + Dy)  
#     w[1:-1,1:-1] = wn[1:-1,1:-1] + dt * rhs

#     # UPDATE VORTICITY BOUNDARY CONDITIONS 
#     w[:,0] = 2.0*(psi_wall - psi[:,1])/dx/dx                # left wall
#     w[:,-1] = 2.0*(psi_wall - psi[:,-2])/dx/dx              # right wall
#     w[0,:] = 2.0*(psi_wall - psi[1,:])/dy/dy                # bottom wall
#     w[-1,:] = 2.0*(psi_wall - psi[-2,:])/dy/dy - 2.0*Ut/dy  # top wall 

#     wsol.append(w)
    
#     t += dt
        


In [ ]:
# plt.contourf(psisol[-1])
# plt.axis('square')
# # plt.savefig('output.pdf', format='pdf', bbox_inches='tight')
# # plt.close()

In [ ]:
# plt.contourf(wsol[-1])
# plt.axis('square')

In [ ]:
# x = np.linspace(0,1,nx)
# y = np.linspace(0,1,ny)
# xx,yy = np.meshgrid(x,y)
# nn = 1
# psi = psisol[-1]
# u = (psi[2:,1:-1] - psi[:-2,1:-1])/2.0/dy
# v = -(psi[1:-1, 2:] - psi[1:-1,:-2])/2.0/dx

# # print(u)
# # print(v)

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(xx[1:-1,1:-1], yy[1:-1,1:-1], np.sqrt(u*u + v*v), levels = 100, cmap=plt.cm.jet)
# ax.streamplot(xx[1:-1,1:-1],yy[1:-1,1:-1],u, v, color=abs(u*u + v*v),cmap=plt.cm.autumn, linewidth=2)
# ax.set_xlim([xx[0,1],xx[0,-2]])
# ax.set_aspect(1)
